In [ ]:

import itertools
import numpy as np
import os
import seaborn as sns
from tqdm import tqdm
from dataclasses import asdict, dataclass, field
import vsketch
import shapely.geometry as sg
from shapely.geometry import box, MultiLineString, Point, MultiPoint, Polygon, MultiPolygon, LineString
import shapely.affinity as sa
import shapely.ops as so
import matplotlib.pyplot as plt
import pandas as pd

import vpype_cli
from typing import List, Generic
from genpen import genpen as gp
from genpen.utils import Paper
from scipy import stats as ss
import geopandas
from shapely.errors import TopologicalError
import functools
import vpype
from skimage import io
from pathlib import Path

from sklearn.preprocessing import minmax_scale
from skimage import feature
from skimage import exposure

from skimage import filters
from skimage.color import rgb2gray
from skimage.transform import rescale, resize, downscale_local_mean
from skimage.morphology import disk
from pyaxidraw import axidraw   # import module

from PIL import Image

import fn
import cv2

from genpen.flow.field import *
from genpen.flow.particle import *
from genpen.genpen import random_line_subdivide_gen

import time

%load_ext autoreload
%autoreload 2

# more complex

In [ ]:
image_path= '/home/naka/art/wigglesphere.jpg'
filename = 'qgrid_flowbeam.svg'
paper_size:str = '6x5.99 inches'
border:float=15  # mm
image_rescale_factor:float=0.04
smooth_disk_size:int=1
hist_clip_limit=0.1
hist_nbins=32
intensity_min=0.
intensity_max=1.
hatch_spacing_min=0.35  # mm
hatch_spacing_max=1.1 # mm
pixel_width=1 # mm
pixel_height=1 # mm
angle_jitter='ss.norm(loc=10, scale=0).rvs'  # degrees
pixel_rotation='0'  # degrees
merge_tolerances=[0.3, 0.4,]  # mm
simplify_tolerances=[0.2,]  # mm
savedir='/home/naka/art/plotter_svgs'

In [ ]:
# make page
paper = Paper(paper_size)
drawbox = paper.get_drawbox(border)


In [ ]:
x0gen = ss.uniform(loc=0.4, scale=0.3).rvs
x1gen = ss.uniform(loc=0.2, scale=0.1).rvs
split_func = functools.partial(gp.split_along_longest_side_of_min_rectangle, xgen=x0gen)
splits = gp.recursive_split_frac_buffer(
    drawbox, 
    split_func=split_func,
    p_continue=0.9, 
    depth=0, 
    depth_limit=10,
    buffer_frac=-0.0
)

bps = MultiPolygon([p for p in splits])

   
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.5mm')
sk.geometry(bps.boundary)

# tolerance=0.5

sk.display()

In [ ]:
all_bps = gp.Shape(bps)

In [ ]:
# make pixel polys
prms = []
for bp in tqdm(bps):

#     a = np.random.uniform(0, 240)
    dist_from_center = bp.centroid.distance(bps.centroid)
    a = np.interp(dist_from_center, [0, 150], [0, 1020])
    prm = {
        'geometry':bp,
        'raw_pixel_width':pixel_width,
        'raw_pixel_height':pixel_height,
        'angle':a,
        'group': 'raw_hatch_pixel',
        'magnitude': np.random.uniform(0.3, 2),

    }
    prms.append(prm)
raw_hatch_pixels = geopandas.GeoDataFrame(prms)

In [ ]:
#  rescale polys to fit in drawbox
bbox = box(*raw_hatch_pixels.total_bounds)
_, transform = gp.make_like(bbox, drawbox, return_transform=True)
A = gp.AffineMatrix(**transform)
scaled_hatch_pixels = raw_hatch_pixels.copy()
scaled_hatch_pixels['geometry'] = scaled_hatch_pixels.affine_transform(A.A_flat)
scaled_hatch_pixels['scaled_pixel_height'] = scaled_hatch_pixels['geometry'].apply(gp.get_height)
scaled_hatch_pixels['scaled_pixel_width'] = scaled_hatch_pixels['geometry'].apply(gp.get_width)

In [ ]:
new_drawbox = so.unary_union(scaled_hatch_pixels.geometry)
db = gp.Poly(new_drawbox)

In [ ]:
# scaled_hatch_pixels['angle'] = np.interp(scaled_hatch_pixels.geometry.centroid.y, [db.bottom, db.top], [0, 680]) + np.random.randn(len(scaled_hatch_pixels)) * 5

In [ ]:
scaled_hatch_pixels['angle'] = scaled_hatch_pixels['angle'] // 5 * 5

In [ ]:
# scaled_hatch_pixels['angle'] = np.interp(scaled_hatch_pixels['angle'], xp=[0, 180], fp=[30, 150])

In [ ]:
qpg = NoisyQuantizedPiecewiseGrid(scaled_hatch_pixels, xstep=5, ystep=5, noise_scale=0.1, noise_mult=0.5, verbose=False)
qpg.make_grid()

In [ ]:
spiral_angle_max = np.pi * 40
spiral_angle_min = 0
spiral_angle_spacing = np.pi * 0.1001
sp_angle_range = np.arange(spiral_angle_min, spiral_angle_max, spiral_angle_spacing)
spiral_distances = np.linspace(0, 100, len(sp_angle_range))

start_points = [Point(np.cos(a) * d, np.sin(a) * d) for a, d in zip(sp_angle_range, spiral_distances)]

In [ ]:
start_points = gp.make_like(MultiPoint(start_points), db.p)

In [ ]:
poly = new_drawbox
pts = []
lss = []
n_steps = 19
for pt in tqdm(start_points):
    
    vp = VectorParticle(pos=pt, grid=qpg, stepsize=1, momentum_factor=np.random.uniform(0,0))
    for ii in range(n_steps):
        vp.step()
    if len(vp.pts) > 1:
        ls = gp.merge_LineStrings([LineString(vp.pts)])
    lss.append(ls)

In [ ]:
buffer_gen = ss.uniform(loc=0.5, scale=0.5).rvs
d_buffer_gen = functools.partial(np.random.uniform, low=-0.35, high=-0.30)
d_translate_factor_gen = ss.uniform(loc=0.6, scale=0.8).rvs

In [ ]:
fills = []
all_polys = Polygon()
for ii, l in enumerate(tqdm(lss[:])):
    p = l.buffer(0.5, cap_style=2, join_style=2)
    p = p.buffer(buffer_gen(), cap_style=2, join_style=2)
    angles_gen = gp.make_callable(sp_angle_range[ii])
#     angles_gen = gp.make_callable(-90)
    stp = gp.ScaleTransPrms(d_buffer=d_buffer_gen(),angles=angles_gen(),d_translate_factor=d_translate_factor_gen(), n_iters=300)
    stp.d_buffers += np.random.uniform(-0.05, 0.05, size=stp.d_buffers.shape)
    P = gp.Poly(p)
    P.fill_scale_trans(**stp.prms)
    visible_area = p.difference(all_polys)
    visible_fill = P.fill.intersection(visible_area.buffer(1e-6))
    
    fills.append(visible_fill)
    all_polys = so.unary_union([all_polys, p])

In [ ]:
blss = gp.merge_LineStrings([f for f in fills if f.length > 0.1])


In [ ]:
blss = blss.intersection(drawbox)

In [ ]:
n_layers = 1

In [ ]:
layers = []
for ii in range(n_layers):
    fills = []
    for p in bps:
        xjitter_func = 0
        yjitter_func = ss.norm(loc=0, scale=np.random.uniform(0.1, 1.5)).rvs
        dist_from_center = bp.centroid.distance(bps.centroid)
        a = np.interp(dist_from_center, [0, 150], [0, 1020])
        bhf = gp.BezierHatchFill(
            spacing=np.random.uniform(0.4, 1.4),
            degrees=a,
            poly_to_fill=p, 
            xjitter_func=xjitter_func, 
            yjitter_func=yjitter_func,
            fill_inscribe_buffer=1.4,
            n_nodes_per_line=15,
            n_eval_points=100,
        )
        fills.append(bhf.p)

    fills = [f for f in fills if f.length > 0]
    layer = gp.merge_LineStrings(fills)
    layers.append(layer)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')
for i, layer in enumerate(layers):
    sk.stroke(i+1)
    sk.geometry(layer)

# sk.stroke(len(layers)+2)
# sk.geometry(blss)
    
sk.display(color_mode='layer')

In [ ]:
filename = '0002_qgrid_bhatch.svg'

In [ ]:
sk.vpype('linemerge -t 0.4 linesimplify linesort')

In [ ]:
savepath = Path(savedir).joinpath(filename).as_posix()
sk.save(savepath)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.stroke(1)
sk.geometry(blss)
    
sk.display()

# circle fluffs

In [ ]:
image_path= '/home/naka/art/wigglesphere.jpg'
filename = 'qgrid_flowbeam.svg'
paper_size:str = '6x5.99 inches'
border:float=15  # mm
image_rescale_factor:float=0.04
smooth_disk_size:int=1
hist_clip_limit=0.1
hist_nbins=32
intensity_min=0.
intensity_max=1.
hatch_spacing_min=0.35  # mm
hatch_spacing_max=1.1 # mm
pixel_width=1 # mm
pixel_height=1 # mm
angle_jitter='ss.norm(loc=10, scale=0).rvs'  # degrees
pixel_rotation='0'  # degrees
merge_tolerances=[0.3, 0.4,]  # mm
simplify_tolerances=[0.2,]  # mm
savedir='/home/naka/art/plotter_svgs'

In [ ]:
# make page
paper = Paper(paper_size)
drawbox = paper.get_drawbox(border)


In [ ]:
bps = gp.circle_pack_within_poly(drawbox, rads=[20, 10, 5, 2.5])

In [ ]:
bps2 = gp.circle_pack_within_poly(drawbox, rads=[20, 10, 5,])

In [ ]:
bps = bps.difference(bps2.boundary.buffer(0.5))

In [ ]:
n_layers = 1

In [ ]:
layers = []
for ii in range(n_layers):
    fills = []
    for p in bps:
        xjitter_func = 0
        yjitter_func = ss.norm(loc=0, scale=np.random.uniform(0.1, 1.5)).rvs
        dist_from_center = p.centroid.distance(bps.centroid)
        a = np.interp(dist_from_center, [0, 150], [0, 1020])
        bhf = gp.BezierHatchFill(
            spacing=np.random.uniform(0.4, 1.4),
            degrees=a,
            poly_to_fill=p, 
            xjitter_func=xjitter_func, 
            yjitter_func=yjitter_func,
            fill_inscribe_buffer=1.4,
            n_nodes_per_line=15,
            n_eval_points=100,
        )
        fills.append(bhf.p)

    fills = [f for f in fills if f.length > 0]
    layer = gp.merge_LineStrings(fills)
    layers.append(layer)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')
for i, layer in enumerate(layers):
    sk.stroke(i+1)
    sk.geometry(layer)

# sk.stroke(len(layers)+2)
# sk.geometry(blss)
    
sk.display(color_mode='layer')

In [ ]:
filename = '0004_circle_bhatch.svg'

In [ ]:
sk.vpype('linemerge -t 0.4 linesimplify linesort')

In [ ]:
savepath = Path(savedir).joinpath(filename).as_posix()
sk.save(savepath)

# circle fluffs

In [ ]:
image_path= '/home/naka/art/wigglesphere.jpg'

paper_size:str = '6x5.99 inches'
border:float=15  # mm
image_rescale_factor:float=0.04
smooth_disk_size:int=1
hist_clip_limit=0.1
hist_nbins=32
intensity_min=0.
intensity_max=1.
hatch_spacing_min=0.35  # mm
hatch_spacing_max=1.1 # mm
pixel_width=1 # mm
pixel_height=1 # mm
angle_jitter='ss.norm(loc=10, scale=0).rvs'  # degrees
pixel_rotation='0'  # degrees
merge_tolerances=[0.3, 0.4,]  # mm
simplify_tolerances=[0.2,]  # mm
savedir='/home/naka/art/plotter_svgs'

In [ ]:
# make page
paper = Paper(paper_size)
drawbox = paper.get_drawbox(border)


In [ ]:
bps = gp.circle_pack_within_poly(drawbox, rads=[20, 10, 5, 2.5])

In [ ]:
bps2 = gp.circle_pack_within_poly(drawbox, rads=[20, 10, 5,])

In [ ]:
bps = bps.difference(bps2.boundary.buffer(0.5))

In [ ]:
n_layers = 1

In [ ]:
layers = []
for ii in range(n_layers):
    fills = []
    for p in bps:
        xjitter_func = 0
        yjitter_func = ss.norm(loc=0, scale=np.random.uniform(0.1, 1.5)).rvs
        dist_from_center = p.centroid.distance(bps.centroid)
        a = np.interp(dist_from_center, [0, 150], [0, 1020])
        bhf = gp.BezierHatchFill(
            spacing=np.random.uniform(0.4, 1.4),
            degrees=a,
            poly_to_fill=p, 
            xjitter_func=xjitter_func, 
            yjitter_func=yjitter_func,
            fill_inscribe_buffer=1.4,
            n_nodes_per_line=15,
            n_eval_points=100,
        )
        fills.append(bhf.p)

    fills = [f for f in fills if f.length > 0]
    layer = gp.merge_LineStrings(fills)
    layers.append(layer)

In [ ]:
mlayers = [layer.buffer(0.001).buffer(-0.001).boundary for layer in layers]

In [ ]:
mlayers = [gp.merge_LineStrings([l for l in layer if l.length > 0.2]) for layer in mlayers]

In [ ]:
sns.displot([np.log10(l.length) for l in mlayers[0]])

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')
for i, layer in enumerate(mlayers):
    sk.stroke(i+1)
    sk.geometry(layer)

# sk.stroke(len(layers)+2)
# sk.geometry(blss)
    
sk.display(color_mode='layer')

In [ ]:
filename = fn.Fn(postfix='-circle_bez_morsed.svg', git_sha_size=5, pid_sha_size=6).name(utc=False, milli=False)

In [ ]:
sk.vpype('splitall linemerge -t 0.1 linesimplify linemerge -t 0.3 linesimplify linemerge -t 0.5 linesimplify linesort')

In [ ]:
sk.display(color_mode='layer')

In [ ]:
savepath = Path(savedir).joinpath(filename).as_posix()
sk.save(savepath)

# particles?

In [ ]:
# make pixel polys
prms = []
for bp in tqdm(bps):

#     a = np.random.uniform(0, 240)
    dist_from_center = bp.centroid.distance(bps.centroid)
    a = np.interp(dist_from_center, [0, 150], [0, 1020])
    prm = {
        'geometry':bp,
        'raw_pixel_width':pixel_width,
        'raw_pixel_height':pixel_height,
        'angle':a,
        'group': 'raw_hatch_pixel',
        'magnitude': np.random.uniform(0.3, 2),

    }
    prms.append(prm)
raw_hatch_pixels = geopandas.GeoDataFrame(prms)

In [ ]:
#  rescale polys to fit in drawbox
bbox = box(*raw_hatch_pixels.total_bounds)
_, transform = gp.make_like(bbox, drawbox, return_transform=True)
A = gp.AffineMatrix(**transform)
scaled_hatch_pixels = raw_hatch_pixels.copy()
scaled_hatch_pixels['geometry'] = scaled_hatch_pixels.affine_transform(A.A_flat)
scaled_hatch_pixels['scaled_pixel_height'] = scaled_hatch_pixels['geometry'].apply(gp.get_height)
scaled_hatch_pixels['scaled_pixel_width'] = scaled_hatch_pixels['geometry'].apply(gp.get_width)

In [ ]:
new_drawbox = so.unary_union(scaled_hatch_pixels.geometry)
db = gp.Poly(new_drawbox)

In [ ]:
qpg = NoisyQuantizedPiecewiseGrid(scaled_hatch_pixels, xstep=5, ystep=5, noise_scale=0.1, noise_mult=1.5, verbose=False)
qpg.make_grid()

In [ ]:
start_points = []
for bp in scaled_hatch_pixels['geometry']:
    start_points += gp.get_random_points_in_polygon(bp, bp.area//4+5) 

In [ ]:
start_points = gp.make_like(MultiPoint(start_points), db.p)

In [ ]:
poly = new_drawbox
pts = []
lss = []
n_steps = 12
for pt in tqdm(start_points):
    
    vp = VectorParticle(pos=pt, grid=qpg, stepsize=1, momentum_factor=np.random.uniform(0,0))
    for ii in range(n_steps):
        vp.step()
    if len(vp.pts) > 1:
        ls = gp.merge_LineStrings([LineString(vp.pts)])
        lss.append(ls)

In [ ]:
lss = gp.merge_LineStrings([f for f in lss if f.length > 0.1])

In [ ]:
split_inds = np.array_split(np.arange(len(lss)), 1000)

In [ ]:
layers = []
for split_ind in split_inds:
    ls = gp.merge_LineStrings([lss[ii] for ii in split_ind])
    layers.append(ls)

In [ ]:
filename = fn.Fn(postfix='-circle_bez_morsed.svg', git_sha_size=5, pid_sha_size=6).name(utc=False, milli=False)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')
# sk.geometry(lss)
# sk.stroke(2)
# sk.geometry(bps.boundary)
for ii, l in enumerate(layers):
    sk.stroke(ii + 1)
    sk.geometry(l)
    
# sk.stroke(len(layers)+2)
# sk.geometry(blss)
    
sk.display(color_mode='layer')

In [ ]:
sk.vpype('splitall linemerge -t 0.1 linesimplify linemerge -t 0.2 linesort')

In [ ]:
sk.display(color_mode='layer')

In [ ]:
savepath = Path(savedir).joinpath(filename).as_posix()
sk.save(savepath)

In [ ]:
n_layers = 1

In [ ]:
sns.displot([p.centroid.distance(bps.centroid) for p in bps])

In [ ]:

fills = []
for p in bps:
    xjitter_func = 0
    yjitter_func = ss.norm(loc=0, scale=np.random.uniform(0.1, 1.5)).rvs
    dist_from_center = p.centroid.distance(bps.centroid)
    a = np.interp(dist_from_center, [0, 80], [0, 180])
    bhf = gp.BezierHatchFill(
        spacing=np.random.uniform(0.3, 0.9),
        degrees=a,
        poly_to_fill=p, 
        xjitter_func=xjitter_func, 
        yjitter_func=yjitter_func,
        fill_inscribe_buffer=1.4,
        n_nodes_per_line=15,
        n_eval_points=100,
    )
    fills.append(bhf.p)

fills = [f for f in fills if f.length > 0]
lss = gp.merge_LineStrings(fills)

In [ ]:
split_inds = np.array_split(np.arange(len(lss)), 1000)

In [ ]:
layers = []
for split_ind in split_inds:
    ls = gp.merge_LineStrings([lss[ii] for ii in split_ind])
    layers.append(ls)

In [ ]:
filename = fn.Fn(postfix='-circle_bez_morsed.svg', git_sha_size=5, pid_sha_size=6).name(utc=False, milli=False)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')
# sk.geometry(lss)
# sk.stroke(2)
# sk.geometry(bps.boundary)
for ii, l in enumerate(layers):
    sk.stroke(ii + 1)
    sk.geometry(l)
    
# sk.stroke(len(layers)+2)
# sk.geometry(blss)
    
sk.display(color_mode='layer')

In [ ]:
sk.vpype('splitall linemerge -t 0.1 linesimplify linemerge -t 0.2 linesort')

In [ ]:
sk.display(color_mode='layer')

In [ ]:
savepath = Path(savedir).joinpath(filename).as_posix()
sk.save(savepath)